<a href="https://colab.research.google.com/github/JinFree/NetworkSplit/blob/main/vgg16_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import numpy as np
import random
from PIL import Image
from tqdm import tqdm

import torch
from torch.utils.data import Dataset
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms
import torchvision.transforms as transforms
from torchvision import models
###   http://www.gisdeveloper.co.kr/?p=8443
SEED_NUM=42
random.seed(a=SEED_NUM, version=2)
np.random.seed(seed=SEED_NUM)
torch.manual_seed(SEED_NUM)
torch.cuda.manual_seed(SEED_NUM)
torch.cuda.manual_seed_all(SEED_NUM)
torch.backends.cudnn.enabled = False
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

def input_trained_params_conv2d(src, dst, is_forward_half = True):
    stride = 0
    if not is_forward_half:
        stride = int(dst.weight.shape[0])
        
    for i in range(dst.weight.shape[0]):
        dst.weight[i] = src.weight[i+stride]
        dst.bias[i] = src.bias[i+stride]
        
    return dst

def input_trained_params_linear(src, dst):
    dst.weight = src.weight
    dst.bias = src.bias
    return dst

In [ ]:
class VGG16_HALF(nn.Module):
    def __init__(self, num_classes = 1000, inference = False):
        super().__init__()
        self.inference = inference
        self.inf_checker = 0
        
        self.conv_0_1 = nn.Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.conv_0_2 = nn.Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.layer_0 = None
        
        self.conv_2_1 = nn.Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.conv_2_2 = nn.Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.layer_2 = None
        
        self.conv_5_1 = nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.conv_5_2 = nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.layer_5 = None
        
        self.conv_7_1 = nn.Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.conv_7_2 = nn.Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.layer_7 = None
        
        
        self.conv_10_1 = nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.conv_10_2 = nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.layer_10 = None
        
        self.conv_12_1 = nn.Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.conv_12_2 = nn.Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.layer_12 = None
        
        self.conv_14_1 = nn.Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.conv_14_2 = nn.Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.layer_14 = None
        
        self.conv_17_1 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.conv_17_2 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.layer_17 = None
        
        self.conv_19_1 = nn.Conv2d(512, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.conv_19_2 = nn.Conv2d(512, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.layer_19 = None
        
        
        self.conv_21_1 = nn.Conv2d(512, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.conv_21_2 = nn.Conv2d(512, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.layer_21 = None
        
        self.conv_24_1 = nn.Conv2d(512, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.conv_24_2 = nn.Conv2d(512, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.layer_24 = None
        
        self.conv_26_1 = nn.Conv2d(512, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.conv_26_2 = nn.Conv2d(512, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.layer_26 = None
        
        self.conv_28_1 = nn.Conv2d(512, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.conv_28_2 = nn.Conv2d(512, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))  
        self.layer_28 = None      
                
        self.result = None
    
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )
    
    def inference_with_torchvision(self):
        network = models.vgg16(pretrained=True)

        input_trained_params_conv2d(network.features[0], self.conv_0_1, True)
        input_trained_params_conv2d(network.features[0], self.conv_0_2, False)

        input_trained_params_conv2d(network.features[2], self.conv_2_1, True)
        input_trained_params_conv2d(network.features[2], self.conv_2_2, False)

        input_trained_params_conv2d(network.features[5], self.conv_5_1, True)
        input_trained_params_conv2d(network.features[5], self.conv_5_2, False)

        input_trained_params_conv2d(network.features[7], self.conv_7_1, True)
        input_trained_params_conv2d(network.features[7], self.conv_7_2, False)

        input_trained_params_conv2d(network.features[10], self.conv_10_1, True)
        input_trained_params_conv2d(network.features[10], self.conv_10_2, False)

        input_trained_params_conv2d(network.features[12], self.conv_12_1, True)
        input_trained_params_conv2d(network.features[12], self.conv_12_2, False)

        input_trained_params_conv2d(network.features[14], self.conv_14_1, True)
        input_trained_params_conv2d(network.features[14], self.conv_14_2, False)

        input_trained_params_conv2d(network.features[17], self.conv_17_1, True)
        input_trained_params_conv2d(network.features[17], self.conv_17_2, False)

        input_trained_params_conv2d(network.features[19], self.conv_19_1, True)
        input_trained_params_conv2d(network.features[19], self.conv_19_2, False)

        input_trained_params_conv2d(network.features[21], self.conv_21_1, True)
        input_trained_params_conv2d(network.features[21], self.conv_21_2, False)

        input_trained_params_conv2d(network.features[24], self.conv_24_1, True)
        input_trained_params_conv2d(network.features[24], self.conv_24_2, False)

        input_trained_params_conv2d(network.features[26], self.conv_26_1, True)
        input_trained_params_conv2d(network.features[26], self.conv_26_2, False)

        input_trained_params_conv2d(network.features[28], self.conv_28_1, True)
        input_trained_params_conv2d(network.features[28], self.conv_28_2, False)

        input_trained_params_linear(network.classifier[0], self.classifier[0])
        input_trained_params_linear(network.classifier[3], self.classifier[3])
        input_trained_params_linear(network.classifier[6], self.classifier[6])
        
    
    def change_status(self):
        if self.inference:
            self.inference = False
        else:
            self.inference = True
        print(self.inference)
        
    def forward(self, x):
        if self.inference:
            if self.inf_checker == 0:
                self.inf_checker += 1
                
                self.layer_0 = self.conv_0_1(x)
                layer_0_2 = self.conv_0_2(x)
                x = torch.cat((self.layer_0, layer_0_2), dim=1)

                x = F.relu(x, inplace=True)

                self.layer_2 = self.conv_2_1(x)
                layer_2_2 = self.conv_2_2(x)
                x = torch.cat((self.layer_2, layer_2_2), dim=1)

                x = F.relu(x, inplace=True)
                x = F.max_pool2d(x, kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


                self.layer_5 = self.conv_5_1(x)
                layer_5_2 = self.conv_5_2(x)
                x = torch.cat((self.layer_5, layer_5_2), dim=1)

                x = F.relu(x, inplace=True)


                self.layer_7 = self.conv_7_1(x)
                layer_7_2 = self.conv_7_2(x)
                x = torch.cat((self.layer_7, layer_7_2), dim=1)

                x = F.relu(x, inplace=True)
                x = F.max_pool2d(x, kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


                self.layer_10 = self.conv_10_1(x)
                layer_10_2 = self.conv_10_2(x)
                x = torch.cat((self.layer_10, layer_10_2), dim=1)

                x = F.relu(x, inplace=True)


                self.layer_12 = self.conv_12_1(x)
                layer_12_2 = self.conv_12_2(x)
                x = torch.cat((self.layer_12, layer_12_2), dim=1)

                x = F.relu(x, inplace=True)


                self.layer_14 = self.conv_14_1(x)
                layer_14_2 = self.conv_14_2(x)
                x = torch.cat((self.layer_14, layer_14_2), dim=1)

                x = F.relu(x, inplace=True)
                x = F.max_pool2d(x, kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


                self.layer_17 = self.conv_17_1(x)
                layer_17_2 = self.conv_17_2(x)
                x = torch.cat((self.layer_17, layer_17_2), dim=1)

                x = F.relu(x, inplace=True)


                self.layer_19 = self.conv_19_1(x)
                layer_19_2 = self.conv_19_2(x)
                x = torch.cat((self.layer_19, layer_19_2), dim=1)

                x = F.relu(x, inplace=True)


                self.layer_21 = self.conv_21_1(x)
                layer_21_2 = self.conv_21_2(x)
                x = torch.cat((self.layer_21, layer_21_2), dim=1)

                x = F.relu(x, inplace=True)
                x = F.max_pool2d(x, kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


                self.layer_24 = self.conv_24_1(x)
                layer_24_2 = self.conv_24_2(x)
                x = torch.cat((self.layer_24, layer_24_2), dim=1)

                x = F.relu(x, inplace=True)


                self.layer_26 = self.conv_26_1(x)
                layer_26_2 = self.conv_26_2(x)
                x = torch.cat((self.layer_26, layer_26_2), dim=1)

                x = F.relu(x, inplace=True)


                self.layer_28 = self.conv_28_1(x)
                layer_28_2 = self.conv_28_2(x)
                x = torch.cat((self.layer_28, layer_28_2), dim=1)

                x = F.relu(x, inplace=True)
                self.result = F.max_pool2d(x, kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
                
            elif self.inf_checker == 1:
                self.inf_checker += 1
                
                layer_0_2 = self.conv_0_2(x)
                x = torch.cat((self.layer_0, layer_0_2), dim=1)
                self.layer_0 = layer_0_2
                x = F.relu(x, inplace=True)

                layer_2_2 = self.conv_2_2(x)
                x = torch.cat((self.layer_2, layer_2_2), dim=1)
                self.layer_2 = layer_2_2
                x = F.relu(x, inplace=True)
                x = F.max_pool2d(x, kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


                layer_5_2 = self.conv_5_2(x)
                x = torch.cat((self.layer_5, layer_5_2), dim=1)
                self.layer_5 = layer_5_2
                x = F.relu(x, inplace=True)


                layer_7_2 = self.conv_7_2(x)
                x = torch.cat((self.layer_7, layer_7_2), dim=1)
                self.layer_7 = layer_7_2
                x = F.relu(x, inplace=True)
                x = F.max_pool2d(x, kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


                layer_10_2 = self.conv_10_2(x)
                x = torch.cat((self.layer_10, layer_10_2), dim=1)
                self.layer_10 = layer_10_2
                x = F.relu(x, inplace=True)


                layer_12_2 = self.conv_12_2(x)
                x = torch.cat((self.layer_12, layer_12_2), dim=1)
                self.layer_12 = layer_12_2
                x = F.relu(x, inplace=True)


                layer_14_2 = self.conv_14_2(x)
                x = torch.cat((self.layer_14, layer_14_2), dim=1)
                self.layer_14  = layer_14_2
                x = F.relu(x, inplace=True)
                x = F.max_pool2d(x, kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


                layer_17_2 = self.conv_17_2(x)
                x = torch.cat((self.layer_17, layer_17_2), dim=1)
                self.layer_17  = layer_17_2
                x = F.relu(x, inplace=True)


                layer_19_2 = self.conv_19_2(x)
                x = torch.cat((self.layer_19, layer_19_2), dim=1)
                self.layer_19 = layer_19_2
                x = F.relu(x, inplace=True)


                layer_21_2 = self.conv_21_2(x)
                x = torch.cat((self.layer_21, layer_21_2), dim=1)
                self.layer_21 = layer_21_2
                x = F.relu(x, inplace=True)
                x = F.max_pool2d(x, kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


                layer_24_2 = self.conv_24_2(x)
                x = torch.cat((self.layer_24, layer_24_2), dim=1)
                self.layer_24 = layer_24_2
                x = F.relu(x, inplace=True)


                layer_26_2 = self.conv_26_2(x)
                x = torch.cat((self.layer_26, layer_26_2), dim=1)
                self.layer_26  = layer_26_2
                x = F.relu(x, inplace=True)


                layer_28_2 = self.conv_28_2(x)
                x = torch.cat((self.layer_28, layer_28_2), dim=1)
                self.layer_28 = layer_28_2
                x = F.relu(x, inplace=True)
                self.result = F.max_pool2d(x, kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
                
            elif self.inf_checker == 2:
                self.inf_checker -= 1
                
                layer_0_1 = self.conv_0_1(x)
                x = torch.cat((layer_0_1, self.layer_0), dim=1)
                self.layer_0 = layer_0_1
                x = F.relu(x, inplace=True)

                layer_2_1 = self.conv_2_1(x)
                x = torch.cat((layer_2_1, self.layer_2), dim=1)
                self.layer_2 = layer_2_1
                x = F.relu(x, inplace=True)
                x = F.max_pool2d(x, kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


                layer_5_1 = self.conv_5_1(x)
                x = torch.cat((layer_5_1, self.layer_5), dim=1)
                self.layer_5 = layer_5_1
                x = F.relu(x, inplace=True)


                layer_7_1 = self.conv_7_1(x)
                x = torch.cat((layer_7_1, self.layer_7), dim=1)
                self.layer_7 = layer_7_1
                x = F.relu(x, inplace=True)
                x = F.max_pool2d(x, kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


                layer_10_1 = self.conv_10_1(x)
                x = torch.cat((layer_10_1, self.layer_10), dim=1)
                self.layer_10 = layer_10_1
                x = F.relu(x, inplace=True)


                layer_12_1 = self.conv_12_1(x)
                x = torch.cat((layer_12_1, self.layer_12), dim=1)
                self.layer_12 = layer_12_1
                x = F.relu(x, inplace=True)


                layer_14_1 = self.conv_14_1(x)
                x = torch.cat((layer_14_1, self.layer_14), dim=1)
                self.layer_14  = layer_14_1
                x = F.relu(x, inplace=True)
                x = F.max_pool2d(x, kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


                layer_17_1 = self.conv_17_1(x)
                x = torch.cat((layer_17_1, self.layer_17), dim=1)
                self.layer_17  = layer_17_1
                x = F.relu(x, inplace=True)


                layer_19_1 = self.conv_19_1(x)
                x = torch.cat((layer_19_1, self.layer_19), dim=1)
                self.layer_19 = layer_19_1
                x = F.relu(x, inplace=True)


                layer_21_1 = self.conv_21_1(x)
                x = torch.cat((layer_21_1, self.layer_21), dim=1)
                self.layer_21 = layer_21_1
                x = F.relu(x, inplace=True)
                x = F.max_pool2d(x, kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


                layer_24_1 = self.conv_24_1(x)
                x = torch.cat((layer_24_1, self.layer_24), dim=1)
                self.layer_24 = layer_24_1
                x = F.relu(x, inplace=True)


                layer_26_1 = self.conv_26_1(x)
                x = torch.cat((layer_26_1, self.layer_26), dim=1)
                self.layer_26  = layer_26_1
                x = F.relu(x, inplace=True)


                layer_28_1 = self.conv_28_1(x)
                x = torch.cat((layer_28_1, self.layer_28), dim=1)
                self.layer_28 = layer_28_1
                x = F.relu(x, inplace=True)
                self.result = F.max_pool2d(x, kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        else:
            layer_0_1 = self.conv_0_1(x)
            layer_0_2 = self.conv_0_2(x)
            x = torch.cat((layer_0_1, layer_0_2), dim=1)
            
            x = F.relu(x, inplace=True)
            
            layer_2_1 = self.conv_2_1(x)
            layer_2_2 = self.conv_2_2(x)
            x = torch.cat((layer_2_1, layer_2_2), dim=1)
            
            x = F.relu(x, inplace=True)
            x = F.max_pool2d(x, kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
            
            
            layer_5_1 = self.conv_5_1(x)
            layer_5_2 = self.conv_5_2(x)
            x = torch.cat((layer_5_1, layer_5_2), dim=1)
            
            x = F.relu(x, inplace=True)
            
            
            layer_7_1 = self.conv_7_1(x)
            layer_7_2 = self.conv_7_2(x)
            x = torch.cat((layer_7_1, layer_7_2), dim=1)
            
            x = F.relu(x, inplace=True)
            x = F.max_pool2d(x, kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
            
            
            layer_10_1 = self.conv_10_1(x)
            layer_10_2 = self.conv_10_2(x)
            x = torch.cat((layer_10_1, layer_10_2), dim=1)
            
            x = F.relu(x, inplace=True)
            
            
            layer_12_1 = self.conv_12_1(x)
            layer_12_2 = self.conv_12_2(x)
            x = torch.cat((layer_12_1, layer_12_2), dim=1)
            
            x = F.relu(x, inplace=True)
            
            
            layer_14_1 = self.conv_14_1(x)
            layer_14_2 = self.conv_14_2(x)
            x = torch.cat((layer_14_1, layer_14_2), dim=1)
            
            x = F.relu(x, inplace=True)
            x = F.max_pool2d(x, kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
            
            
            layer_17_1 = self.conv_17_1(x)
            layer_17_2 = self.conv_17_2(x)
            x = torch.cat((layer_17_1, layer_17_2), dim=1)
            
            x = F.relu(x, inplace=True)
            
            
            layer_19_1 = self.conv_19_1(x)
            layer_19_2 = self.conv_19_2(x)
            x = torch.cat((layer_19_1, layer_19_2), dim=1)
            
            x = F.relu(x, inplace=True)
            
            
            layer_21_1 = self.conv_21_1(x)
            layer_21_2 = self.conv_21_2(x)
            x = torch.cat((layer_21_1, layer_21_2), dim=1)
            
            x = F.relu(x, inplace=True)
            x = F.max_pool2d(x, kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
            
            
            layer_24_1 = self.conv_24_1(x)
            layer_24_2 = self.conv_24_2(x)
            x = torch.cat((layer_24_1, layer_24_2), dim=1)
            
            x = F.relu(x, inplace=True)
            
            
            layer_26_1 = self.conv_26_1(x)
            layer_26_2 = self.conv_26_2(x)
            x = torch.cat((layer_26_1, layer_26_2), dim=1)
            
            x = F.relu(x, inplace=True)
            
            
            layer_28_1 = self.conv_28_1(x)
            layer_28_2 = self.conv_28_2(x)
            x = torch.cat((layer_28_1, layer_28_2), dim=1)
            
            x = F.relu(x, inplace=True)
            self.result = F.max_pool2d(x, kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
            
            
            
        x = self.avgpool(self.result)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [ ]:
network = models.vgg16(pretrained=True)
network

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
network.features[0].bias

Parameter containing:
tensor([ 0.4034,  0.3778,  0.4644, -0.3228,  0.3940, -0.3953,  0.3951, -0.5496,
         0.2693, -0.7602, -0.3508,  0.2334, -1.3239, -0.1694,  0.3938, -0.1026,
         0.0460, -0.6995,  0.1549,  0.5628,  0.3011,  0.3425,  0.1073,  0.4651,
         0.1295,  0.0788, -0.0492, -0.5638,  0.1465, -0.3890, -0.0715,  0.0649,
         0.2768,  0.3279,  0.5682, -1.2640, -0.8368, -0.9485,  0.1358,  0.2727,
         0.1841, -0.5325,  0.3507, -0.0827, -1.0248, -0.6912, -0.7711,  0.2612,
         0.4033, -0.4802, -0.3066,  0.5807, -1.3325,  0.4844, -0.8160,  0.2386,
         0.2300,  0.4979,  0.5553,  0.5230, -0.2182,  0.0117, -0.5516,  0.2108],
       requires_grad=True)

In [ ]:
network2 = VGG16_HALF()
network2.inference_with_torchvision()

In [ ]:
network2.conv_0_2.bias

Parameter containing:
tensor([ 0.2768,  0.3279,  0.5682, -1.2640, -0.8368, -0.9485,  0.1358,  0.2727,
         0.1841, -0.5325,  0.3507, -0.0827, -1.0248, -0.6912, -0.7711,  0.2612,
         0.4033, -0.4802, -0.3066,  0.5807, -1.3325,  0.4844, -0.8160,  0.2386,
         0.2300,  0.4979,  0.5553,  0.5230, -0.2182,  0.0117, -0.5516,  0.2108],
       grad_fn=<CopySlices>)

In [ ]:
import os
import cv2
from PIL import Image
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import numpy as np

USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

test_tensor = torch.rand(1, 3, 244, 244)

In [ ]:
test_tensor = test_tensor.to(DEVICE)
network = network.to(DEVICE).eval()
network2 = network2.to(DEVICE).eval()

In [ ]:
import time

def eval(model, tensor):
    time_ = time.time()
    output = None
    with torch.no_grad():
        time_ -= time_
        for i in range(10000):
            start = time.time()  # 시작 시간 저장
            output = model(tensor)
            end = time.time() - start
            time_ += end
    print(output)
    print("time :", time_)  # 현재시각 - 시작시간 = 실행 시간
    return

In [ ]:
eval(network, test_tensor)

tensor([[-7.4385e-01,  1.4804e+00, -5.3962e-01,  5.4345e-01, -4.1696e-01,
         -1.8613e-01, -6.0308e-01, -4.6140e-01, -5.9823e-01,  8.8839e-03,
          4.5382e-01,  1.2223e+00,  9.3680e-01,  1.0030e+00,  7.3924e-01,
          5.4289e-01,  2.3086e-01, -2.5446e-01,  1.1991e-01,  8.6787e-01,
          2.4375e-01,  6.2169e-01,  4.5873e-02, -3.5269e-01,  4.0353e-01,
         -6.8085e-01,  7.5815e-01, -1.4649e-03, -1.1122e+00,  2.7171e-01,
         -1.1803e+00, -5.9074e-02, -4.6356e-01, -4.3567e-01, -3.3285e-01,
         -8.0615e-01,  2.6849e-01, -1.7941e+00,  4.1561e-01, -1.3428e+00,
         -1.1536e-02, -3.6764e-01, -1.6046e-01,  1.2216e-01, -6.0874e-01,
         -7.0940e-01, -2.6716e-01, -3.0937e-01, -1.1674e+00, -2.4726e-01,
          5.9597e-01, -1.2327e+00,  2.5826e-01,  5.8163e-01, -1.1394e-01,
         -4.9212e-01, -4.9626e-01, -1.1175e+00,  1.0810e+00,  4.5191e-01,
         -1.1261e-01, -1.3682e+00, -7.1520e-01,  8.7955e-01,  7.0190e-01,
          4.8610e-01,  2.2557e-02, -4.

In [ ]:
network2.change_status()
eval(network2, test_tensor)

True
tensor([[-7.4385e-01,  1.4804e+00, -5.3962e-01,  5.4345e-01, -4.1696e-01,
         -1.8613e-01, -6.0308e-01, -4.6140e-01, -5.9823e-01,  8.8838e-03,
          4.5382e-01,  1.2223e+00,  9.3680e-01,  1.0030e+00,  7.3924e-01,
          5.4289e-01,  2.3086e-01, -2.5446e-01,  1.1991e-01,  8.6787e-01,
          2.4375e-01,  6.2169e-01,  4.5873e-02, -3.5269e-01,  4.0353e-01,
         -6.8085e-01,  7.5815e-01, -1.4653e-03, -1.1122e+00,  2.7171e-01,
         -1.1803e+00, -5.9074e-02, -4.6356e-01, -4.3567e-01, -3.3285e-01,
         -8.0615e-01,  2.6849e-01, -1.7941e+00,  4.1561e-01, -1.3428e+00,
         -1.1537e-02, -3.6764e-01, -1.6046e-01,  1.2216e-01, -6.0874e-01,
         -7.0940e-01, -2.6716e-01, -3.0937e-01, -1.1674e+00, -2.4726e-01,
          5.9597e-01, -1.2327e+00,  2.5826e-01,  5.8163e-01, -1.1394e-01,
         -4.9212e-01, -4.9626e-01, -1.1175e+00,  1.0810e+00,  4.5191e-01,
         -1.1261e-01, -1.3682e+00, -7.1520e-01,  8.7955e-01,  7.0190e-01,
          4.8610e-01,  2.2557e-02

In [ ]:
network2.change_status()
eval(network2, test_tensor)

False
tensor([[-7.4385e-01,  1.4804e+00, -5.3962e-01,  5.4345e-01, -4.1696e-01,
         -1.8613e-01, -6.0308e-01, -4.6140e-01, -5.9823e-01,  8.8838e-03,
          4.5382e-01,  1.2223e+00,  9.3680e-01,  1.0030e+00,  7.3924e-01,
          5.4289e-01,  2.3086e-01, -2.5446e-01,  1.1991e-01,  8.6787e-01,
          2.4375e-01,  6.2169e-01,  4.5873e-02, -3.5269e-01,  4.0353e-01,
         -6.8085e-01,  7.5815e-01, -1.4653e-03, -1.1122e+00,  2.7171e-01,
         -1.1803e+00, -5.9074e-02, -4.6356e-01, -4.3567e-01, -3.3285e-01,
         -8.0615e-01,  2.6849e-01, -1.7941e+00,  4.1561e-01, -1.3428e+00,
         -1.1537e-02, -3.6764e-01, -1.6046e-01,  1.2216e-01, -6.0874e-01,
         -7.0940e-01, -2.6716e-01, -3.0937e-01, -1.1674e+00, -2.4726e-01,
          5.9597e-01, -1.2327e+00,  2.5826e-01,  5.8163e-01, -1.1394e-01,
         -4.9212e-01, -4.9626e-01, -1.1175e+00,  1.0810e+00,  4.5191e-01,
         -1.1261e-01, -1.3682e+00, -7.1520e-01,  8.7955e-01,  7.0190e-01,
          4.8610e-01,  2.2557e-0